### Analisis de Sentimiento a tweets en Español con el clasificador Support Vector Machine

#### Tweets obtenidos de base de datos con tweets recolectados en español de usuarios con geolocalizacion en Guatemala

### tweets class
* 0 = negativo
* 1 = positivo
* 2 = neutral

### Imports:

In [1]:
import MySQLdb
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

### Retrieves data from db:

In [28]:
#Retrieve tweets from db
conn = MySQLdb.connect("13.58.190.139","root","123","tesis" )
data = pd.read_sql("select * from tweets where class is not null", conn)
data_copy = data

### Split data:

In [29]:
#Split label from dataset
y = data_copy["class"]
X = data_copy["text"]

#Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5958,), (1987,), (5958,), (1987,))

In [20]:
#Create dummy indicators for column label
y = pd.get_dummies(y, columns=["class"], prefix="class")
#Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y)


### Import stop words:

In [31]:
# Import spanish stopword
spanish_stopwords = stopwords.words('spanish')
# Spanish stemmer
stemmer = SnowballStemmer('spanish')
analyzer = CountVectorizer(stop_words = spanish_stopwords).build_analyzer()

In [32]:
# Applies stemmer function to text
def customized_analyzer(doc):
    stemmed_doc = []
    for text in doc:
        word_list = ''
        for word in analyzer(text):
            item = str(stemmer.stem(word))
            word_list = word_list + " " + item
        stemmed_doc.append(word_list)
    return stemmed_doc

In [65]:
# Import spanish stopword
spanish_stopwords = stopwords.words('spanish')

vectorizer = CountVectorizer(
                analyzer = 'word',
                lowercase = True,
                ngram_range = (1,3),
                stop_words = spanish_stopwords)

In [66]:
# Bag of Words from training set
X_train_counts = vectorizer.fit_transform((X_train))

In [67]:
# Train classifier with TF-IDF words weigth
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [68]:
X_new_counts = vectorizer.transform((X_test))
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

### Build pipeline for classifier

In [71]:
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42)),
 ])
text_clf.fit(X_train, y_train)  
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test) 

0.65676899849018622

In [72]:
# Print results
print(metrics.classification_report(y_test, predicted, target_names=("Negativo","Positivo","Neutral")))
#metrics.confusion_matrix(y_test, predicted)

             precision    recall  f1-score   support

   Negativo       0.17      0.01      0.01       180
   Positivo       0.44      0.34      0.38       501
    Neutral       0.71      0.87      0.78      1306

avg / total       0.59      0.66      0.61      1987



### Check accuracy for different SVM classifiers

In [86]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.8,
                             sublinear_tf=True,stop_words = spanish_stopwords,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

# Perform classification with SVM, kernel=rbf
classifier_rbf = svm.SVC()
t0 = time.time()
classifier_rbf.fit(X_train_counts, y_train)
t1 = time.time()
prediction_rbf = classifier_rbf.predict(X_new_counts)
t2 = time.time()
time_rbf_train = t1-t0
time_rbf_predict = t2-t1

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(X_train_counts, y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(X_new_counts)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# Perform classification with SVM, kernel=linear
classifier_liblinear = svm.LinearSVC()
t0 = time.time()
classifier_liblinear.fit(X_train_counts, y_train)
t1 = time.time()
prediction_liblinear = classifier_liblinear.predict(X_new_counts)
t2 = time.time()
time_liblinear_train = t1-t0
time_liblinear_predict = t2-t1

# Print results in a nice table
print("RESULTS FOR SVC(kernel=rbf)")
print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
print(classification_report(y_test, prediction_rbf, target_names=("Negativo","Positivo","Neutral")))
print("RESULTS FOR SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(y_test, prediction_linear, target_names=("Negativo","Positivo","Neutral")))
print("RESULTS FOR LinearSVC()")
print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
print(classification_report(y_test, prediction_liblinear, target_names=("Negativo","Positivo","Neutral")))

RESULTS FOR SVC(kernel=rbf)
Training time: 1.030719s; Prediction time: 0.224569s
             precision    recall  f1-score   support

   Negativo       0.00      0.00      0.00       180
   Positivo       0.12      0.00      0.00       501
    Neutral       0.66      1.00      0.79      1306

avg / total       0.46      0.66      0.52      1987

RESULTS FOR SVC(kernel=linear)
Training time: 0.333240s; Prediction time: 0.059819s
             precision    recall  f1-score   support

   Negativo       0.00      0.00      0.00       180
   Positivo       0.00      0.00      0.00       501
    Neutral       0.66      1.00      0.79      1306

avg / total       0.43      0.66      0.52      1987

RESULTS FOR LinearSVC()
Training time: 0.834805s; Prediction time: 0.000213s
             precision    recall  f1-score   support

   Negativo       0.00      0.00      0.00       180
   Positivo       0.00      0.00      0.00       501
    Neutral       0.66      1.00      0.79      1306

avg / to

/home/sara/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [76]:
# Test Linear model using TF-IDF::

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(X_train_tfidf, y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(X_new_tfidf)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

print("RESULTS FOR SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(y_test, prediction_linear, target_names=("Negativo","Positivo","Neutral")))

RESULTS FOR SVC(kernel=linear)
Training time: 0.290675s; Prediction time: 0.059952s
             precision    recall  f1-score   support

   Negativo       0.00      0.00      0.00       180
   Positivo       0.00      0.00      0.00       501
    Neutral       0.66      1.00      0.79      1306

avg / total       0.43      0.66      0.52      1987



/home/sara/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [77]:
# Score del classificador
classifier_linear.score(X_new_tfidf, y_test) #0.69112

0.65727226975339703

In [87]:
print("RESULTS FOR SVC(kernel=rbf)",classifier_rbf.score(X_new_counts, y_test))
print("RESULTS FOR SVC(kernel=linear)",classifier_linear.score(X_new_counts, y_test))
print("RESULTS FOR LinearSVC()",classifier_liblinear.score(X_new_counts, y_test))

RESULTS FOR SVC(kernel=rbf) 0.655762455964
RESULTS FOR SVC(kernel=linear) 0.657272269753
RESULTS FOR LinearSVC() 0.657272269753


### NOTAS:
* El SVM con kernel lineal es el que da mejor accuracy 
* Usar TF-IDF incrementa el accuracy

In [64]:
#Create dummy indicators for column label
#y = pd.get_dummies(y, columns=["class"], prefix="class")
#Split dataset into training and test set
#X_train, X_test, y_train, y_test = train_test_split(X, y)


'''y = label_binarize(y, classes=[0, 1, 2, 3])
clf_SVM = OneVsRestClassifier(LinearSVC())
params = {
      'estimator__C': [0.5, 1.0, 1.5],
      'estimator__tol': [1e-3, 1e-4, 1e-5],
      }
gs = GridSearchCV(clf_SVM, params, cv=5, scoring='roc_auc')
gs.fit(corpus1, y)'''


from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('vect', vectorizer),
    ('cls', svm.LinearSVC()),
])

parameters = {
    'vect__max_df': (0.5, 1.9),
    'vect__min_df': (10, 20,50),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'cls__C': (0.2, 0.5, 0.7),
    'cls__loss': ('hinge', 'squared_hinge'),
    'cls__max_iter': (500, 1000)
}


grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1 , scoring='roc_auc')
grid_search.fit(X_train, y_tparsed)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/sara/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/sara/anaconda3/lib/python

In [63]:
y_tparsed = pd.get_dummies(y_train, columns=["class"], prefix="class")

In [66]:
y_tparsed.head()

,class_0,class_1,class_2
318,0,0,1
816,0,0,1
2636,0,0,1
59,0,1,0
3257,0,0,1
